### Implements algorithm 1 in the parti-game [paper](https://link.springer.com/content/pdf/10.1007/BF00993591.pdf)

Learn a controller from a start region to a goal region on a continuous space; Four increasingly effective algorithms to partition discrete state spaces. Algorithm 1 and 2 are non-learning; 3 and 4 learn, hence explore, the world while planning a route to the goal. Here, I implement algorithm 1

In [1]:
import sys
import numpy as np
from os.path import abspath, join
sys.path.append(abspath(join('..')))
sys.path.append(abspath(join('..', 'grids')))
sys.path.append(abspath(join('..', 'utils')))
sys.path.append(abspath(join('..', 'Visualization')))
sys.path.append(abspath(join('..')))

from grids import *
import matplotlib.pyplot as plt
from gen_utils import *
from BoundaryCondition import *
from Visualization import Visualizer
from valFuncs import proj
%matplotlib inline

In [94]:
#Given N cells, construct an MDT with N discrete states 1, ..., N.

N = 6

# make a 6 cell grid
gridIn=expand(np.array((0, 1, 0, 1, 0, 1)), 1)
gridOut =expand(np.array((1, 2, 1, 2, 1, 2)), 1)
num_points = 30
N = num_points*ones(6,1).astype(np.int64)

g = createGrid(gridIn, gridOut, N, process=True, low_mem=False);

# print(f'len(g.xs), g.xs[0].shape {len(g.xs), g.xs[0].shape} g.N {g.N.shape}')
dims = [[0, 4], [1, 5], [2, 3]]

bounds = [np.linspace(0, 1, 3), np.linspace(1, 2, 4), np.linspace(2, 3, 3), np.linspace(3, 4, 4)]
        
if g.dim > 1:
    bounds_grid = np.meshgrid(*bounds, sparse=False);
else:
    bounds_grid = np.meshgrid(bounds)[0]

Grid dimension > 5, may be dangerously large


In [111]:
N = 4

# make a 6 cell grid
gridIn=expand(np.array((0, 0)), 1)
gridOut =expand(np.array((1, 1)), 1)
num_points = 30
N = num_points*ones(2,1).astype(np.int64)

g = createGrid(gridIn, gridOut, N, process=True, low_mem=False);

bounds = [[0, 0.5, 1], [0, 0.25, 0.75, 1]]
padding = np.array(([[0.1, 0.2]])).T

if g.dim > 1:
    bounds_grid = np.meshgrid(*bounds, sparse=False, indexing='ij');
else:
    bounds_grid = np.meshgrid(bounds)[0]

bds_grd = bounds_grid[0]
bds_grd_shp = np.array(bds_grd.shape)

gs = bds_grd_shp-(bds_grd_shp>1)
gs = np.empty(gs)

In [118]:

for i in range(numel(gs)):
    ii = np.unravel_index(size(gs), i, order='F')
    pritn(f'ii: {ii}')
end

ValueError: index 2 is out of bounds for array with size 0

In [120]:
help(np.unravel_index)

Help on function unravel_index in module numpy:

unravel_index(...)
    unravel_index(indices, shape, order='C')
    
    Converts a flat index or array of flat indices into a tuple
    of coordinate arrays.
    
    Parameters
    ----------
    indices : array_like
        An integer array whose elements are indices into the flattened
        version of an array of dimensions ``shape``. Before version 1.6.0,
        this function accepted just one index value.
    shape : tuple of ints
        The shape of the array to use for unraveling ``indices``.
    
        .. versionchanged:: 1.16.0
            Renamed from ``dims`` to ``shape``.
    
    order : {'C', 'F'}, optional
        Determines whether the indices should be viewed as indexing in
        row-major (C-style) or column-major (Fortran-style) order.
    
        .. versionadded:: 1.6.0
    
    Returns
    -------
    unraveled_coords : tuple of ndarray
        Each array in the tuple has the same shape as the ``indices``
 